# Python MDCS Class

This notebook is designed to demonstrate the simple functionality of the Python mdcs.MDCS class.

## Library Imports

In [1]:
import os
import time

#This imports the mdcs-api Python tools
import mdcs

## Initialize MDCS Link

Interacting with a curator requires certain access information.  The MDCS class helps with this by allowing this information to be defined once, and does a quick check that the information is valid.

Keyword arguments for initilization:

- host = root url for the curator.

- user = username, or tuple consisting of (username, password).

- pswd = password.

- cert = certification file that may be necessary for some curators.

- records_fetch = boolean indicating if a local copy of all records are built during initialization. Default value is True.

#### Initialize an empty MDCS instance and fill in values separately

In [2]:
empty = mdcs.MDCS()
empty.host = 'http://127.0.0.1:8000/'
empty.user = ('admin', 'admin')

#### Initilize a link to a local MDCS using password prompt

In [3]:
local =  mdcs.MDCS(host='http://127.0.0.1:8000/', user='admin')

Enter password for admin on http://127.0.0.1:8000/:
password:········


#### Initilize a link to a remote MDCS using both certification and password files

In [4]:
iprhub = mdcs.MDCS(host='https://iprhub.nist.gov/', 
                   user='lmh1',
                   pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                   cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem')

ValueError: Certification file not found!

## Explore Types and Templates

When initialized with access information, the MDCS object will query the database and build local Pandas DataFrames of all the types and templates. Copies of these DataFrames can be accessed using the MDCS.templates and MDCS.xsd_types properties.

#### List templates attribute names 

In [5]:
for key in local.templates:
    print key

XSLTFiles
content
dependencies
exporters
filename
hash
id
templateVersion
title
version


In [6]:
local.templates

,XSLTFiles,content,dependencies,exporters,filename,hash,id,templateVersion,title,version
0,[],"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",[57b3502398a5e11d98150185],[],record-lammps-potential.xsd,23c20caab03fe167efc727c977cd66fafbe96ea2,57b3502598a5e11d98150188,57b3502598a5e11d98150187,LAMMPS-potential,1


#### List names of all templates on local

In [7]:
for template in local.templates.title.tolist():
    print template

LAMMPS-potential


#### List names of all types on local

In [8]:
for xsd_type in local.xsd_types.title.tolist():
    print xsd_type

interatomic-potential-compact-id
chemical-element
lammps-potential


### Finding a Specific Template or Type

Pandas has great built-in tools for parsing through the listed records. For convience, the MDCS class also has the methods get_template() and get_xsd_type() for locating a specific template or type. These methods return the Series of the matching template/type and issue an error if exactly one matching template/type is not found. 

One optional argument is allowed. If given, a limiting search is performed using the template's title and filename (if not specified as keywords).

If any of the following keyword arguments are given, the returned template must have matching corresponding values for the terms.

- filename = name of the file associated with the template/type

- title = name assigned to the template/type

- version = version assigned to the template/type

#### Get template information for LAMMPS-potential template

In [9]:
local.get_template('LAMMPS-potential')

XSLTFiles                                                         []
content            <?xml version="1.0" encoding="UTF-8" standalon...
dependencies                              [57b3502398a5e11d98150185]
exporters                                                         []
filename                                 record-lammps-potential.xsd
hash                        23c20caab03fe167efc727c977cd66fafbe96ea2
id                                          57b3502598a5e11d98150188
templateVersion                             57b3502598a5e11d98150187
title                                               LAMMPS-potential
version                                                            1
Name: 0, dtype: object

## Explore Records

Records are similarly copied to a local DataFrame and can be viewed with the property MDCS.records. 

#### Display the content of iprhub.records

In [ ]:
iprhub.records

### Finding a Specific Record

The get_record() method can be used for locating a specific record. Returns the Series of the matching record. Issues an error if exactly one matching record is not found. 

If any of the following keyword arguments are given, the returned template must have matching corresponding values for the terms.

Keyword arguments are:

- title = name or id associated with the record.

- template = template (schema) associated with the record. Can be title, filename, id or Series representation of a template.

#### Call get_record with title only

In [ ]:
iprhub.get_record('1987--Ackland-G-J--Cu')

#### Call get_record with title and template

In [ ]:
iprhub.get_record(title =    'potential.2015--Mendelev-M-I--Al-Sm', 
                  template = 'record-interatomic-potential')

## Refresh the Local Information

The DataFrames for the types, templates and records can be rebuilt by calling the refresh method. This is useful if external changes are made to the curator.

In [10]:
local.refresh()

## Adding Templates and Types

Templates and types can be easily added using the add_template() and add_xsd_type() methods.  Both functions work nearly identically.

Keyword arguments for both methods:

- filename = path to file being loaded.

- title = name to assign to template/type. If not given, then the filename minus extension is used.

- version = version number.

- dependencies = list of types that are required by the file being loaded. Each item can be identified by the type's filename, id, title, or as a Pandas Series.

- refresh = Boolean indicating if refresh() is called automatically. Default is True.

It will not add if the title and version of the template/type match with an existing template/type.

#### Call add_xsd_type with an already existing title and version

In [11]:
local.add_xsd_type('demo_files/uncertainty.xsd', title='uncertainty')

uploading uncertainty


#### Call add_xsd_type again with a new name

In [12]:
local.add_xsd_type('demo_files/uncertainty.xsd', title='newt-uncertainty')

uploading newt-uncertainty


#### Show that new type exists

In [13]:
local.get_xsd_type('newt-uncertainty')

content         <?xml version="1.0" encoding="UTF-8" standalon...
dependencies                                                   []
filename                                          uncertainty.xsd
hash                     3db5ac62b1aaf1d844e630de15aa0a84b37da39a
id                                       58601f1698a5e11bc84d7f72
title                                            newt-uncertainty
typeVersion                              58601f1698a5e11bc84d7f71
version                                                         1
Name: 4, dtype: object

## Deleting and Restoring Templates and Types

There are also methods delete_template(), delete_xsd_type(), restore_template(), restore_xsd_type() that delete and restore templates and types, but these behaviors don't do much with the current version of the curator.

## Adding Records

A record can be added using the add_record() method. 

Arguments of the function are:

- content = string representation of an xml record, or the path to an xml file.

- title = name to give the record.

- template = schema template that the record is validated against.  Can be title, filename, id or Series representation of a template.

Keyword argument:

- refresh = Boolean indicating if refresh() is called automatically. Default is True.

#### Add a record

In [14]:
local.add_record('demo_files/1987--Ackland-G-J--Ag.xml', '1987--Ackland-G-J--Ag', 'LAMMPS-potential')
local.records

,_id,content,schema,title
0,57b3506b98a5e11d9815018a,"<?xml version=""1.0"" encoding=""utf-8""?>\n<LAMMP...",57b3502598a5e11d98150188,1985--Foiles-S-M--Ni-Cu
1,58601f3198a5e11bc84d7f73,"<?xml version=""1.0"" encoding=""utf-8""?>\n<LAMMP...",57b3502598a5e11d98150188,1987--Ackland-G-J--Ag


## Modifying Records

Records can be modified with the update_record() method. Note that the corresponding \_id will change as this deletes the old record and adds a new one in its place.

Arguments:

- record = existing record to replace. This can be the record's title, \_id, or a Series representation.

- content =  string representation of an xml record, or the path to an xml file with the new content.

Keyword argument:

- refresh = Boolean indicating if refresh() is called automatically. Default is True.

#### View existing content

In [15]:
record = local.get_record('1987--Ackland-G-J--Ag').content
print record

<?xml version="1.0" encoding="utf-8"?>
<LAMMPS-potential><potential><key>dc4149ce-3592-4131-8683-ecf654d5a519</key><id>1987--Ackland-G-J--Ag</id></potential><units>metal</units><atom_style>atomic</atom_style><atom><element>Ag</element><mass>107.8682</mass></atom><pair_style><type>eam/fs</type></pair_style><pair_coeff><term><file>Ag.eam.fs</file></term><term><symbols>True</symbols></term></pair_coeff></LAMMPS-potential>


#### Modify content and update

In [16]:
new_record = record.replace('<mass>107.8682</mass>', '<mass>107.9</mass>')
local.update_record('1987--Ackland-G-J--Ag', new_record)
local.records

,_id,content,schema,title
0,57b3506b98a5e11d9815018a,"<?xml version=""1.0"" encoding=""utf-8""?>\n<LAMMP...",57b3502598a5e11d98150188,1985--Foiles-S-M--Ni-Cu
1,58601f3a98a5e11bc84d7f74,"<?xml version=""1.0"" encoding=""utf-8""?>\n<LAMMP...",57b3502598a5e11d98150188,1987--Ackland-G-J--Ag


#### Check that content has changed

In [17]:
record = local.get_record('1987--Ackland-G-J--Ag').content
print record

<?xml version="1.0" encoding="utf-8"?>
<LAMMPS-potential><potential><key>dc4149ce-3592-4131-8683-ecf654d5a519</key><id>1987--Ackland-G-J--Ag</id></potential><units>metal</units><atom_style>atomic</atom_style><atom><element>Ag</element><mass>107.9</mass></atom><pair_style><type>eam/fs</type></pair_style><pair_coeff><term><file>Ag.eam.fs</file></term><term><symbols>True</symbols></term></pair_coeff></LAMMPS-potential>


## Deleting A Record

An unwanted record can be deleted using the delete_record() method.

Arguments:

- record = existing record to delete. This can be the record's title, \_id, or a Series representation.

Keyword argument:

- refresh = Boolean indicating if refresh() is called automatically. Default is True.

#### Delete the recently added record

In [18]:
local.delete_record('1987--Ackland-G-J--Ag')
local.records

,_id,content,schema,title
0,57b3506b98a5e11d9815018a,"<?xml version=""1.0"" encoding=""utf-8""?>\n<LAMMP...",57b3502598a5e11d98150188,1985--Foiles-S-M--Ni-Cu


## Add Other Files (blobs)

Non-record files can also be stored on the curator as blobs. These can be added with the add_file() method.

Argument:

- filename = path to the file to upload.

Returns the url that can be used to download the file later.

In [19]:
file_url = local.add_file('demo_files/test.txt')
print file_url

http://127.0.0.1:8000/rest/blob?id=58601f4098a5e11bc84d7f75


## Large Repository Tips

When working with large repositories or a large number of records, it would be inefficient to download all records and to refresh everytime records are added or changed. This section details the options that can be done to reduce the local overhead involved for those cases.

### Initialize without building records

Setting records_fetch=False during initialization will build the templates/types but not the records. 

__NOTE:__ accessing records will cause it to be built if it doesn't exist yet.

#### Reinitialize iprhub

In [20]:
start = time.time()
iprhub = mdcs.MDCS(host='https://iprhub.nist.gov/', 
                   user='lmh1',
                   pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                   cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem')
end = time.time()
print end-start, 'seconds'

ValueError: Certification file not found!

#### Reinitialize iprhub without loading records

In [ ]:
start = time.time()
iprhub = mdcs.MDCS(host='https://iprhub.nist.gov/', 
                   user='lmh1',
                   pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                   cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem',
                   build_records=False)
end = time.time()
print end-start, 'seconds'

### Build records with limiters

The build_records() method can be used to then build the records DataFrame with only a selection of the curator's records.

Keyword arguments:

- format = file format to access. Values are xml or json.

- id = record id to limit to.

- template = template of records to include. Can be title, filename, id or Series representation of a template.

- title = record title to limit to.

Given the current options, this is really only good for curators with a large number of templates in that you can specifically limit your search to one template.

#### Retrieve only records for the record-interatomic-potential template

In [ ]:
iprhub.build_records(template='record-interatomic-potential')
iprhub.records

#### Retrieve only records for the record-JARVIS-FF-calc template

In [ ]:
iprhub.build_records(template='record-JARVIS-FF-calc')
iprhub.records

#### The selection information is retained for when refresh() is called

In [ ]:
iprhub.refresh()
iprhub.records

### Supressing refreshes

If a large number of files are being added/modified at once, it doesn't make much sense to refresh the local copies for every small change. All the methods that modify the curator have the keyword refresh which can be set to False to avoid needless refreshing.

__NOTE:__ Many of the capabilities of this class search over the local representations of the templates/types/records, so refreshing is necessary in some cases. Some examples are:

- The get methods will fail if they are searching for items that have been added after the last refresh. 

- Any method that takes templates/types as inputs won't be able to find them if they were added since the last refresh.

- The add_xml_types() and add_templates() may miss duplicate entries if added before a refresh.

#### Demonstrate adding a record without refreshing

In [ ]:
local.records

In [ ]:
local.add_record('demo_files/1987--Ackland-G-J--Ag.xml', '1987--Ackland-G-J--Ag', 'LAMMPS-potential', refresh=False)
local.records

#### Refresh

In [ ]:
local.refresh()
local.records

#### Demonstrate deleting a record without refreshing

In [ ]:
local.delete_record('1987--Ackland-G-J--Ag', refresh=False)
local.records

#### Refresh

In [ ]:
local.refresh()
local.records